<a href="https://colab.research.google.com/github/Forestf90/NoduleDetector007/blob/master/NoduleDetector007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NoduleDetector007

In [ ]:
import numpy as np
import sys
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_nb = r'/content/drive/My Drive/'
sys.path.append(path_nb)

In [ ]:
outfolder = '/content/drive/MyDrive/out/LungNoduleDetectionClassification/0001_0120/'
datafolder = outfolder+'processeddata'
weightsfolder = outfolder+'modelweights'

noduleimages=np.load(datafolder+"/noduleimages_0001_0120.npy")
nodulemasks=np.load(datafolder+"/nodulemasks_0001_0120.npy")

nodulemasks = nodulemasks.astype(float)



In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.layers import Input, Concatenate, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc


K.set_image_data_format('channels_first')
tf.keras.backend.set_image_data_format('channels_first')
tf.keras.backend.image_data_format()

In [ ]:
nodulesize=[np.sum(mask) for mask in nodulemasks]
plt.hist([nod for nod in nodulesize if nod<300],bins=50)
plt.xlabel("Area")
plt.ylabel("frequency")
plt.show()


In [ ]:
if K.image_data_format()=='channels_last':
    noduleimages=noduleimages.reshape(noduleimages.shape[0],512,512,1)
    nodulemasks=nodulemasks.reshape(nodulemasks.shape[0],512,512,1)
else:
    noduleimages=noduleimages.reshape(noduleimages.shape[0],1,512,512)
    nodulemasks=nodulemasks.reshape(nodulemasks.shape[0],1,512,512)


imagestrain, imagestest, maskstrain, maskstest = train_test_split(noduleimages,nodulemasks,test_size=.20)

maskstrain.shape



In [ ]:

num_test=imagestest.shape[0]
num_train=imagestrain.shape[0]

num_test=10
num_train=10
if K.image_data_format()=='channels_last':
    for index in range(num_test):
        print("Ground Truth test")
        plt.imshow(np.squeeze(maskstest[index]),cmap="gray")
        plt.show()
        print("Image test")
        plt.imshow(np.squeeze(imagestest[index]), cmap="gray")
        plt.show()
            
    for index in range(num_test):
        print("Ground Truth train")
        plt.imshow(np.squeeze(maskstrain[index]),cmap="gray")
        plt.show()
        print("Image train")
        plt.imshow(np.squeeze(imagestrain[index]), cmap="gray")
        plt.show()
else:
    for index in range(num_test):
        print("Ground Truth test")
        plt.imshow(maskstest[index,0],cmap="gray")
        plt.show()
        print("Image test")
        plt.imshow(imagestest[index,0], cmap="gray")
        plt.show()
            
    for index in range(num_test):
        print("Ground Truth train")
        plt.imshow(maskstrain[index,0],cmap="gray")
        plt.show()
        print("Image train")
        plt.imshow(imagestrain[index,0], cmap="gray")
        plt.show()



In [ ]:
#Code sourced from https://www.kaggle.com/c/data-science-bowl-2017#tutorial
smooth = 1.0
width = 32

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


In [ ]:
from tensorflow.keras import layers


def get_model(img_size, num_classes):
    
    if K.image_data_format()=='channels_first':
        input_shape = (1,512, 512)
        print((1,512, 512))
    else:
        input_shape = (512, 512,1)
        print((512, 512,1))

    inputs = Input(input_shape)

    ### [First half of the network: downsampling inputs] ###
    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
K.clear_session()

# Build model
model = get_model((512,512), 1)
model.summary()

In [ ]:
import tensorflow as tf
filepath="/content/drive/My Drive/Colab Notebooks/MS.hdf5"

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef, 'accuracy'])


checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True)
history=model.fit(imagestrain, maskstrain, batch_size=4, epochs=10, verbose=1, shuffle=True,
              callbacks=[checkpoint],validation_data=(imagestest,maskstest))

In [ ]:
history
print(history.history.keys())
plt.plot(history.history['dice_coef'], color='b')
plt.plot(history.history['val_dice_coef'], color='g')
plt.xlabel("Epoch")
plt.ylabel("Dice Coefficient")
plt.legend(["Train", "Validation"])
plt.show()


plt.plot(history.history['accuracy'], color='b')
plt.plot(history.history['val_accuracy'], color='g')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Train", "Validation"])
plt.show()

plt.plot(history.history['loss'], color='b')
plt.plot(history.history['val_loss'], color='g')
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.legend(["Train", "Validation"])
plt.show()

In [ ]:
model.evaluate(imagestest,maskstest, batch_size=4)

In [ ]:
imagestest.shape


In [ ]:

if K.image_data_format()=='channels_last':

    num_test=imagestest.shape[0]
    imgs_mask_test = np.ndarray([num_test,512,512,1],dtype=np.float32)

    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imagestest[i:i+1]], verbose=0)[0]

    print(imgs_mask_test.shape)

    sumoverlap=[]
    for i in range(num_test):
        sumoverlap.append(np.sum(maskstest[i]*imgs_mask_test[i]))

    print(len([ov for ov in sumoverlap if ov>1])/len(sumoverlap))

else:
    num_test=imagestest.shape[0]
    imgs_mask_test = np.ndarray([num_test,1,512,512],dtype=np.float32)

    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imagestest[i:i+1]], verbose=0)[0]


    sumoverlap=[]
    for i in range(num_test):
        sumoverlap.append(np.sum(maskstest[i,0]*imgs_mask_test[i,0]))

    print(len([ov for ov in sumoverlap if ov>1])/len(sumoverlap))

In [ ]:
index=1

if K.image_data_format()=='channels_last':
    print("Predicted")
    plt.imshow(np.squeeze(imgs_mask_test[index]), cmap="gray")
    plt.show()
    print("Ground Truth")
    plt.imshow(np.squeeze(maskstest[index]),cmap="gray")
    plt.show()
    print("Image")
    plt.imshow(np.squeeze(imagestest[index]), cmap="gray")
    plt.show()
else:
    print("Predicted")
    plt.imshow(imgs_mask_test[index,0], cmap="gray")
    plt.show()
    print("Ground Truth")
    plt.imshow(maskstest[index,0],cmap="gray")
    plt.show()
    print("Image")
    plt.imshow(imagestest[index,0], cmap="gray")
    plt.show()


